<a href="https://colab.research.google.com/github/g4aidl-upc-winter-2020/3D-Shape-classification/blob/main/GNN_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install all needed packages from PyG:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-cluster -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

     |████████████████████████████████| 2.6MB 7.7MB/s 
     |████████████████████████████████| 1.5MB 7.8MB/s 
     |████████████████████████████████| 1.0MB 6.6MB/s 
     |████████████████████████████████| 194kB 7.2MB/s 
     |████████████████████████████████| 235kB 25.6MB/s 
     |████████████████████████████████| 2.2MB 26.1MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 


In [2]:
import torch
from torch_geometric.datasets import ModelNet
from torch_geometric.data import DataLoader
from torch_geometric.utils import to_dense_batch
import torch_geometric.transforms as T
from torch_geometric.transforms import SamplePoints, KNNGraph, NormalizeScale, Compose

import matplotlib.pyplot as plt

import sys

from sklearn.metrics import confusion_matrix

## Set a fixed seed

In [3]:
seed = 42

#Controlling sources of randomness
torch.manual_seed(seed)  #Sets the seed for generating random numbers for all devices (both CPU and CUDA)

#CUDA convolution benchmarking
torch.backends.cudnn.benchmark = False #ensures that CUDA selects the same algorithm each time an application is run


## Hyper-parameters

In [4]:
test_batch_size = 32
graph_type = 'GAT'        # 'GCN', 'GAT'  

### Import drive folder

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# PointNet

## Dataset

In [6]:
# Import ModelNet10 dataset from PyG
test_dataset = ModelNet(root='/content/drive/MyDrive/Proyecto/Colabs/ModelNet', name="10", train=False, pre_transform=T.SamplePoints(num=1024))  #test dataset

In [7]:
print('Dataset info:')
print('--------------')
print('Test dataset size: ', len(test_dataset))

Dataset info:
--------------
Test dataset size:  908


### Graph Generation and Normalize Input Features

In [8]:
#Input features normalization
test_dataset.transform = Compose([NormalizeScale(), KNNGraph(k=9, loop=True, force_undirected=True)]) #Creates a k-NN undirected graph based on node positions pos

print(f'Dataset: {test_dataset}:')
print('====================')
print(f'Number of testing graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')

Dataset: ModelNet10(908):
Number of testing graphs: 908
Number of features: 0
Number of classes: 10


This dataset provides **908 different graphs**, and the task is to classify each graph into **one out of 10 classes**

In [9]:
data = test_dataset[0] # Get the first test graph object.
print(data)
print('=============================================================')

# Gather some statistics about the previous graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is directed: {data.is_directed()}')

Data(edge_index=[2, 10744], pos=[1024, 3], y=[1])
Number of nodes: 1024
Number of edges: 10744
Average node degree: 10.49
Contains isolated nodes: False
Contains self-loops: True
Is directed: False


By inspecting the first graph object of the train dataset, we can see that it comes with **1024 nodes (with 3-dimensional spatial vectors)** and **10744 edges** (leading to an average node degree of 9). It also comes with exactly **one graph label** (`y=[1]`).

## Testing

### Make sure your runtime has a GPU

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
assert not device.type == 'cpu', "Change Runtime Type -> GPU"

### Loading the model architecture

In [11]:
# import the architecture class from a python script

if graph_type == 'GCN':
  
  #We include a new file path that will point to modules the we want to import
  sys.path.append('/content/drive/MyDrive/Proyecto/Colabs/architectures/GCN')

  ## Possibilities 
  #from GCN_Architecture_BatchNorm_AVGpool import GCN
  #from GCN_Architecture_BatchNorm_MAXpool import GCN
  from GCN_Architecture_BatchNorm_DoubleCapacity_MAXpool import GCN
  #from GCN_Architecture_BatchNorm_DoubleCapacity_Dropout_MAXpool import GCN
  ##
  model = GCN()                     # instantiate the model
  
else:
  #We include a new file path that will point to modules the we want to import
  sys.path.append('/content/drive/MyDrive/Proyecto/Colabs/architectures/GAT')  

  ## Possibilities
  #from GAT_Architecture_BatchNorm_4heads_MAXpool import GAT
  #from GAT_Architecture_BatchNorm_2heads_MAXpool import GAT
  from GAT_Architecture_BatchNorm_8heads_MAXpool import GAT
  #from GAT_Architecture_BatchNorm_8heads_MAXpool_AVG import GAT
  #from GAT_Architecture_BatchNorm_8heads_MAXpool_Dropout import GAT
  ##
  model = GAT()

model.to(device)   # Pass the model to GPU(device)

GAT(
  (conv1): GATConv(3, 16, heads=8)
  (conv2): GATConv(128, 32, heads=8)
  (conv3): GATConv(256, 64, heads=8)
  (fc): Linear(in_features=512, out_features=10, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

### Loading best parameters from training 

In [12]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Proyecto/Colabs/experiments/logs/GAT/GAT_InitLR0.001_8heads_RandomFlip (concat)/train/best_params.pt'))  #Change directory to load the best params of the model

<All keys matched successfully>

### Dataloader

In [13]:
test_loader = DataLoader(test_dataset, batch_size= test_batch_size)

### Test

In [14]:
model.eval()
with torch.no_grad():
  total=correct=0
  all_preds = []
  all_labels = []
  for data in test_loader:
      
      output = model(data.pos.to(device), data.edge_index.to(device), data.batch.to(device))  
      
      _, preds = torch.max(output.to(device), 1) # We get the maximum prediction value (correct category) for each pointcloud in the batch
      
      total += data.y.size(0)  # total number of samples in the test_loader
      
      correct += (preds == data.y.to(device)).sum().item()  #number of total correct predictions in the test_loader
      
      all_preds += list(preds.cpu().numpy())
      all_labels += list(data.y.cpu().numpy())

val_acc = 100. * (correct / total)
print('Test accuracy: {:.2f} %'.format(val_acc))

Test accuracy: 90.09 %


##Metrics

### Confusion Matrix

In [15]:
#ClassesNames = ['Bathtub', 'Bed', 'Chair', 'Desk', 'Dresser', 'Monitor', 'Night_Stand', 'Sofa', 'Table', 'Toilet']
#Classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [16]:
cm =confusion_matrix(all_labels, all_preds)
cm

array([[ 42,   5,   0,   0,   0,   0,   0,   0,   2,   1],
       [  0,  97,   1,   1,   0,   0,   0,   0,   1,   0],
       [  0,   0, 100,   0,   0,   0,   0,   0,   0,   0],
       [  0,   1,   0,  73,   3,   0,   1,   3,   5,   0],
       [  0,   0,   0,   1,  72,   1,  11,   0,   1,   0],
       [  0,   0,   1,   0,   0,  98,   1,   0,   0,   0],
       [  0,   0,   0,   0,  16,   1,  64,   0,   5,   0],
       [  0,   0,   0,   0,   2,   0,   1,  97,   0,   0],
       [  0,   0,   0,  23,   0,   0,   1,   0,  76,   0],
       [  0,   0,   1,   0,   0,   0,   0,   0,   0,  99]])

### Precision and Recall

In [17]:
##Our implementation of computing Precision and recall scores per class

dim = cm.shape[0]
ClassesNames = ['Bathtub', 'Bed', 'Chair', 'Desk', 'Dresser', 'Monitor', 'Night_Stand', 'Sofa', 'Table', 'Toilet']
# Precision = TP / (TP + FP)
# Recall = TP / (TP + FN)
Precision = []
Recall = []
Correct = 0
Samples = 0
PrecisionWAvg = 0
RecallWAvg = 0
for i in range(0, dim):
  TP = cm[i,i]  #Diagonal value (TP)
  FPc = 0
  FNc = 0
  for j in range(0, dim):
    Samples += cm[i,j]
    FNc += cm[i,j]  #Add all line values
    FPc += cm[j,i]  #Add all column values
  FN = FNc - TP   #Substract diagonal value (TP)
  FP = FPc - TP   #Substract diagonal value (TP)
  Correct += TP
  if TP==0:
    Precision.append(0)
    Recall.append(0)
  else:  
    Precision.append(100*(TP/(TP+FP)))
    Recall.append(100*(TP/(TP+FN)))
    PrecisionWAvg+=100*(TP/(TP+FP))*(TP+FN)
    RecallWAvg+=100*(TP/(TP+FN))*(TP+FN)

  print(ClassesNames[i], "\n\tPrecision: {:.2f}% \tRecall: {:.2f}%".format(Precision[i],Recall[i]))
  print("\tTP:", TP,"  FP:", FP,"  FN:", FN,"  Samples in Test:",TP+FN)

print("\nTOTAL Accuracy: {:.2f}%".format(100*Correct/Samples),"  Samples in Test: ", Samples,"  Correct Predictions: ", Correct)
print("Avg. Weighted Precision: {:.2f}% \tAvg. Weighted Recall: {:.2f}%\n".format(PrecisionWAvg/Samples, RecallWAvg/Samples))


Bathtub 
	Precision: 100.00% 	Recall: 84.00%
	TP: 42   FP: 0   FN: 8   Samples in Test: 50
Bed 
	Precision: 94.17% 	Recall: 97.00%
	TP: 97   FP: 6   FN: 3   Samples in Test: 100
Chair 
	Precision: 97.09% 	Recall: 100.00%
	TP: 100   FP: 3   FN: 0   Samples in Test: 100
Desk 
	Precision: 74.49% 	Recall: 84.88%
	TP: 73   FP: 25   FN: 13   Samples in Test: 86
Dresser 
	Precision: 77.42% 	Recall: 83.72%
	TP: 72   FP: 21   FN: 14   Samples in Test: 86
Monitor 
	Precision: 98.00% 	Recall: 98.00%
	TP: 98   FP: 2   FN: 2   Samples in Test: 100
Night_Stand 
	Precision: 81.01% 	Recall: 74.42%
	TP: 64   FP: 15   FN: 22   Samples in Test: 86
Sofa 
	Precision: 97.00% 	Recall: 97.00%
	TP: 97   FP: 3   FN: 3   Samples in Test: 100
Table 
	Precision: 84.44% 	Recall: 76.00%
	TP: 76   FP: 14   FN: 24   Samples in Test: 100
Toilet 
	Precision: 99.00% 	Recall: 99.00%
	TP: 99   FP: 1   FN: 1   Samples in Test: 100

TOTAL Accuracy: 90.09%   Samples in Test:  908   Correct Predictions:  818
Avg. Weighted Prec